<h1>9장 멀티모달 대규모 언어 모델</h1>
<i>대규모 언어 모델에게 비전 능력을 추가하기</i>

<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter09.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 9장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

In [ ]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
import os
import tqdm
from transformers.utils import logging

# tqdm 비활성화
os.environ["DISABLE_TQDM"] = "1"

logging.disable_progress_bar()

## OpenCLIP

In [ ]:
from urllib.request import urlopen
from PIL import Image

# AI로 생성한 이미지를 로드합니다.
puppy_path = "https://bit.ly/4jYqmPu"
image = Image.open(urlopen(puppy_path)).convert("RGB")
caption = "a puppy playing in the snow"

In [ ]:
image

### 임베딩

In [ ]:
from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel

model_id = "openai/clip-vit-base-patch32"

# 텍스트 전처리를 위한 토크나이저를 로드합니다.
clip_tokenizer = CLIPTokenizerFast.from_pretrained(model_id)

# 이미지 전처리를 위한 전처리기를 로드합니다.
clip_processor = CLIPProcessor.from_pretrained(model_id)

# 텍스트 임베딩과 이미지 임베딩을 생성하기 위한 메인 모델
model = CLIPModel.from_pretrained(model_id)

In [ ]:
# 입력을 토큰으로 나눕니다.
inputs = clip_tokenizer(caption, return_tensors="pt")
inputs

In [ ]:
# 입력 아이디를 토큰으로 되돌립니다.
clip_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

In [ ]:
# 텍스트 임베딩을 만듭니다.
text_embedding = model.get_text_features(**inputs)
text_embedding.shape

In [ ]:
# 이미지를 전처리합니다.
processed_image = clip_processor(
    text=None, images=image, return_tensors='pt'
)['pixel_values']

processed_image.shape

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# 시각화를 이해 이미지를 준비합니다.
img = processed_image.squeeze(0)
img = img.permute(*torch.arange(img.ndim - 1, -1, -1))
img = np.einsum('ijk->jik', img)

# 전처리된 이미지를 출력합니다.
plt.imshow(img)
plt.axis('off')

In [ ]:
# 이미지 임베딩을 만듭니다.
image_embedding = model.get_image_features(processed_image)
image_embedding.shape

In [ ]:
# 임베딩을 정규화합니다.
text_embedding /= text_embedding.norm(dim=-1, keepdim=True)
image_embedding /= image_embedding.norm(dim=-1, keepdim=True)

# 유사도를 계산합니다.
text_embedding = text_embedding.detach().cpu().numpy()
image_embedding = image_embedding.detach().cpu().numpy()
score = text_embedding @ image_embedding.T
score

### 여러 이미지로 유사도 점수 계산하기

In [ ]:
from urllib.request import urlopen
from PIL import Image

# AI로 생성한 이미지를 로드합니다.
cat_path = "https://bit.ly/42UJXJu"
car_path = "https://bit.ly/4cR4rHs"
paths = [puppy_path, cat_path, car_path]
images = [Image.open(urlopen(path)).convert("RGBA") for path in paths]
captions = [
    "a puppy playing in the snow",
    "a pixelated image of a cute cat",
    "A supercar on the road \nwith the sunset in the background"
]

import numpy as np

# 모든 이미지의 임베딩을 만듭니다.
image_embeddings = []
for image in images:
  image_processed = clip_processor(images=image, return_tensors='pt')['pixel_values']
  image_embedding = model.get_image_features(image_processed).detach().cpu().numpy()[0]
  image_embeddings.append(image_embedding)
image_embeddings = np.array(image_embeddings)

# 모든 캡션의 임베딩을 만듭니다.
text_embeddings = []
for caption in captions:
  inputs = clip_tokenizer(caption, return_tensors="pt")
  text_emb = model.get_text_features(**inputs).detach().cpu().numpy()[0]
  text_embeddings.append(text_emb)
text_embeddings = np.array(text_embeddings)

In [ ]:
# 이미지와 캡션의 코사인 유사도를 계산합니다.
from sklearn.metrics.pairwise import cosine_similarity
sim_matrix = cosine_similarity(image_embeddings, text_embeddings)

In [ ]:
# 피겨 객체를 만듭니다.
plt.figure(figsize=(20, 14))
plt.imshow(sim_matrix, cmap='viridis')

# 정답 레이블로 틱을 설정합니다.
plt.yticks(range(len(captions)), captions, fontsize=18)
plt.xticks([])

# 이미지를 출력합니다.
for i, image in enumerate(images):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")

# 캡션을 추가합니다.
for x in range(sim_matrix.shape[1]):
    for y in range(sim_matrix.shape[0]):
        plt.text(x, y, f"{sim_matrix[y, x]:.2f}", ha="center", va="center", size=30)

# 불필요한 요소를 제거합니다.
for side in ["left", "top", "right", "bottom"]:
  plt.gca().spines[side].set_visible(False)

# 크기를 조정합니다.
plt.xlim([-0.5, len(captions) - 0.5])
plt.ylim([len(captions) + 0.5, -2])
# plt.title("Similarity Matrix", size=20)
plt.savefig("sim_matrix.png", dpi=300, bbox_inches='tight')

### SBERT

In [ ]:
from sentence_transformers import SentenceTransformer, util

# SBERT 호환 CLIP 모델을 로드합니다.
model = SentenceTransformer('clip-ViT-B-32')

# 이미지를 인코딩합니다.
image_embeddings = model.encode(images)

# 캡션을 인코딩합니다.
text_embeddings = model.encode(captions)

# 코사인 유사도를 계산합니다.
sim_matrix = util.cos_sim(image_embeddings, text_embeddings)
print(sim_matrix)

## BLIP-2

In [ ]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

# 전처리기와 메인 모델을 로드합니다.
# # Choose specific model because of: https://huggingface.co/Salesforce/blip2-opt-2.7b/discussions/39
blip_processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    torch_dtype=torch.float16
)

# 추론 속도를 높이기 위해 모델을 GPU에 전송합니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

### 이미지 전처리

In [ ]:
# 수퍼카 이미지를 로드합니다.
car_path = "https://bit.ly/4cR4rHs"
image = Image.open(urlopen(car_path)).convert("RGB")
image

In [ ]:
# 이미지를 전처리합니다.
inputs = blip_processor(image, return_tensors="pt").to(device, torch.float16)
inputs["pixel_values"].shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def show_image(image_inputs):
    # 넘파이 배열로 변환하고, 크기를 (1, 3, 224, 224)에서 (224, 224, 3)로 바꿉니다.
    image_inputs = inputs["pixel_values"][0].detach().cpu().numpy()
    image_inputs = np.einsum('ijk->kji', image_inputs)
    image_inputs = np.einsum('ijk->jik', image_inputs)

    # RGB 값에 해당하는 0-255 범위로 변환합니다.
    scaler = MinMaxScaler(feature_range=(0, 255))
    image_inputs = scaler.fit_transform(image_inputs.reshape(-1, image_inputs.shape[-1])).reshape(image_inputs.shape)
    image_inputs = np.array(image_inputs, dtype=np.uint8)

    # 넘파이 배열을 Image 객체로 바꿉니다.
    return Image.fromarray(image_inputs)

show_image(inputs)

### 텍스트 전처리

In [ ]:
blip_processor.tokenizer

In [ ]:
# 텍스트를 전처리합니다.
text = "Her vocalization was remarkably melodic"
token_ids = blip_processor(text=text, return_tensors="pt")
token_ids = token_ids.to(device, torch.float16)["input_ids"][0]

# 입력 ID를 토큰으로 되돌립니다.
tokens = blip_processor.tokenizer.convert_ids_to_tokens(token_ids)
tokens

In [ ]:
# 특수 토큰을 밑줄 문자로 바꿉니다.
tokens = [token.replace("Ġ", "_") for token in tokens]
tokens

### 사용 사례 1: 이미지 캡셔닝

In [ ]:
# AI가 생성한 수퍼카 이미지를 로드합니다.
image = Image.open(urlopen(car_path)).convert("RGB")

# 이미지를 전처리하여 입력을 준비합니다.
inputs = blip_processor(image, return_tensors="pt").to(device, torch.float16)
show_image(inputs)

In [ ]:
# 이미지를 임베딩을 만들고 Q-포머의 출력을 디코더(LLM)에 전달해 토큰 ID를 생성합니다.
generated_ids = model.generate(**inputs, max_new_tokens=20)

# 토큰 ID를 바탕으로 텍스트를 생성합니다.
generated_text = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)
generated_text = generated_text[0].strip()
generated_text

In [ ]:
url = "https://bit.ly/3GJmrra"
image = Image.open(urlopen(url)).convert("RGB")
image

In [ ]:
# 로르샤흐 이미지를 로드합니다.
url = "https://bit.ly/3GJmrra"
image = Image.open(urlopen(url)).convert("RGB")

# 캡션을 생성합니다.
inputs = blip_processor(image, return_tensors="pt").to(device, torch.float16)
generated_ids = model.generate(**inputs, max_new_tokens=20)
generated_text = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)
generated_text = generated_text[0].strip()
generated_text

### 사용 사례 2: 채팅 기반 멀티모달 프롬프트

In [ ]:
# AI로 생성한 수퍼카 이미지를 로드합니다.
image = Image.open(urlopen(car_path)).convert("RGB")

In [ ]:
# 시각 질문 답변
prompt = "Question: Write down what you see in this picture. Answer:"

# 이미지와 프롬프트를 모두 전처리합니다.
inputs = blip_processor(image, text=prompt, return_tensors="pt").to(device, torch.float16)

# 텍스트를 생성합니다.
generated_ids = model.generate(**inputs, max_new_tokens=30)
generated_text = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)
generated_text = generated_text[0].strip()
generated_text

In [ ]:
# 채팅 스타일의 프롬프트
prompt = "Question: Write down what you see in this picture. Answer: A sports car driving on the road at sunset. Question: What would it cost me to drive that car? Answer:"

# 출력을 생성합니다.
inputs = blip_processor(image, text=prompt, return_tensors="pt").to(device, torch.float16)
generated_ids = model.generate(**inputs, max_new_tokens=30)
generated_text = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)
generated_text = generated_text[0].strip()
generated_text

In [ ]:
from IPython.display import HTML, display
import ipywidgets as widgets

def text_eventhandler(*args):
  question = args[0]["new"]
  if question:
    args[0]["owner"].value = ""

    # 프롬프트를 만듭니다.
    if not memory:
      prompt = " Question: " + question + " Answer:"
    else:
      template = "Question: {} Answer: {}."
      prompt = " ".join(
          [
              template.format(memory[i][0], memory[i][1])
              for i in range(len(memory))
          ]
      ) + " Question: " + question + " Answer:"

    # 텍스트를 생성합니다.
    inputs = blip_processor(image, text=prompt, return_tensors="pt")
    inputs = inputs.to(device, torch.float16)
    generated_ids = model.generate(**inputs, max_new_tokens=100)
    generated_text = blip_processor.batch_decode(
        generated_ids,
        skip_special_tokens=True
    )
    generated_text = generated_text[0].strip().split("Answer: ")[-1]

    # 메모리를 업데이트합니다.
    memory.append((question, generated_text))

    # 출력에 할당합니다.
    output.append_display_data(HTML("<b>USER:</b> " + question))
    output.append_display_data(HTML("<b>BLIP-2:</b> " + generated_text))
    output.append_display_data(HTML("<br>"))

# 위젯을 준비합니다.
in_text = widgets.Text()
in_text.continuous_update = False
in_text.observe(text_eventhandler, "value")
output = widgets.Output()
memory = []

# 채팅 상자를 출력합니다.
display(
    widgets.VBox(
        children=[output, in_text],
        layout=widgets.Layout(display="inline-flex", flex_flow="column-reverse"),
    )
)